# Задание 1 Загрузка данных

Изучить представленный набор данных на основе описания его столбцов в файле “horse_data.names” , загрузить его и оставить 8 столбцов для дальнейшего изучения: surgery?, Age, rectal temperature, pulse, respiratory rate, temperature of extremities, pain, outcome.

In [37]:
import pandas as pd

# данные из horse_data.names
column_headers = {
    1: 'surgery?',
    2: 'Age',
    4: 'rectal_temperature',
    5: 'pulse',
    6: 'respiratory_rate',
    7: 'temperature_of_extremities',
    11: 'pain',
    23: 'outcome'
}
# сдвигаем на единицу влево (индекс с 0 начинается)
column_headers = {k-1: v for k, v in column_headers.items()}
# Загрузка данных
df = pd.read_csv('horse_data.csv', header=None)
# Получаем индексы колонок, которые нужно оставить
columns_to_keep = list(column_headers.keys())
# Удаляем колонки, индексы которых не в columns_to_keep
df = df.loc[:, df.columns.isin(columns_to_keep)]
# Присваиваем названия оставшимся колонкам
df.columns = [column_headers[i] for i in df.columns]

# Причесываем типы
# Замена всех вхождений '?' на None
df.replace('?', None, inplace=True)
# Замена всех типов на integer вместо string
for col in df.columns:
    # кроме температуры, она будет float
    if col == 'rectal_temperature':
        df[col] = df[col].astype(float)
    else:
        df[col] = df[col].astype('Int64')

print(df.dtypes)
print()
print(df.head())

surgery?                        Int64
Age                             Int64
rectal_temperature            float64
pulse                           Int64
respiratory_rate                Int64
temperature_of_extremities      Int64
pain                            Int64
outcome                         Int64
dtype: object

   surgery?  Age  rectal_temperature  pulse  respiratory_rate  \
0         2    1                38.5     66                28   
1         1    1                39.2     88                20   
2         2    1                38.3     40                24   
3         1    9                39.1    164                84   
4         2    1                37.3    104                35   

   temperature_of_extremities  pain  outcome  
0                           3     5        2  
1                        <NA>     3        3  
2                           1     3        1  
3                           4     2        2  
4                        <NA>  <NA>        2  


# Задание 2. Первичное изучение данных
Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.

In [39]:
# Оценка структуры данных
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery?                    299 non-null    Int64  
 1   Age                         300 non-null    Int64  
 2   rectal_temperature          240 non-null    float64
 3   pulse                       276 non-null    Int64  
 4   respiratory_rate            242 non-null    Int64  
 5   temperature_of_extremities  244 non-null    Int64  
 6   pain                        245 non-null    Int64  
 7   outcome                     299 non-null    Int64  
dtypes: Int64(7), float64(1)
memory usage: 20.9 KB


In [56]:
# Анализируем выбросы колонки "ректальная температура". Тут хотя бы числа, остальные колонки - категории.
# странно анализировать отсутствие информации на предмет выборосов; игнорируем пустышки
df_cleaned = df.dropna(subset=['rectal_temperature'])
# Средняя температура
print(f"Средняя температура: {df_cleaned['rectal_temperature'].mean()}")
# тык от 35 до 40 градусов (common sense... ходят слухи, что белок в клетках сворачивается при температуре выше 40 градусов)
print(df_cleaned[(df_cleaned['rectal_temperature'] > 35) & (df_cleaned['rectal_temperature'] < 40)]['rectal_temperature'].mean())
print()
# медиана
print(f"Медиана: {df_cleaned['rectal_temperature'].median()}")
print(df_cleaned[(df_cleaned['rectal_temperature'] > 35) & (df_cleaned['rectal_temperature'] < 40)]['rectal_temperature'].median())
print()
# мода
print(f"Мода: {df_cleaned['rectal_temperature'].round().mode()[0]}")
print(df_cleaned[(df_cleaned['rectal_temperature'] > 35) & (df_cleaned['rectal_temperature'] < 40)]['rectal_temperature'].round().mode()[0])
print()

# Выбросы через межквартильный размах
q1 = df_cleaned['rectal_temperature'].quantile(0.25)
q3 = df_cleaned['rectal_temperature'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr)
upper_bound = q3 + (1.5 * iqr)
remove_outliers = df_cleaned['rectal_temperature'].between(lower_bound, upper_bound, inclusive='both')
# что это за выбросы?
df_cleaned[~df_cleaned['rectal_temperature'].between(lower_bound, upper_bound, inclusive='both')]

Средняя температура: 38.16791666666667
38.13093220338982

Медиана: 38.2
38.150000000000006

Мода: 38.0
38.0



,surgery?,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
20,1,1,39.9,72,60,1,5,1
44,1,1,35.4,140,24,3,4,3
54,2,1,40.3,114,36,3,2,3
75,1,9,39.7,100,<NA>,3,2,3
80,1,1,36.4,98,35,3,4,2
91,2,1,40.3,114,36,3,2,2
99,2,1,39.6,108,51,3,2,1
118,1,1,36.5,78,30,1,5,1
141,2,1,36.0,42,30,<NA>,<NA>,2
238,2,1,36.1,88,<NA>,3,3,3
